# 1. Loading necessary variables

In [27]:
project_path = catalog.load('params:project_path') # Project root path
server_ip = catalog.load('params:tpn.server_ip') # IP Address for TPN File Server
domain_name = catalog.load('params:tpn.domain_name') # Domain Name for TPN File Server
shared_folder = catalog.load('params:tpn.shared_folder') # TPN File Server shared folder
sgs_base_path = catalog.load('params:sgs.base_path') # SGS base path on TPN File Server

2021-02-23 20:27:10,742 - kedro.io.data_catalog - INFO - Loading data from `params:project_path` (MemoryDataSet)...
2021-02-23 20:27:10,750 - kedro.io.data_catalog - INFO - Loading data from `params:tpn.server_ip` (MemoryDataSet)...
2021-02-23 20:27:10,752 - kedro.io.data_catalog - INFO - Loading data from `params:tpn.domain_name` (MemoryDataSet)...
2021-02-23 20:27:10,754 - kedro.io.data_catalog - INFO - Loading data from `params:tpn.shared_folder` (MemoryDataSet)...
2021-02-23 20:27:10,756 - kedro.io.data_catalog - INFO - Loading data from `params:sgs.base_path` (MemoryDataSet)...


/home/user01/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 2. Credentials loading Node

In [25]:
import os
from typing import Dict
from kedro.config import ConfigLoader

def get_credentials(project_path: str) -> Dict:
    conf_paths = [
        os.path.join(project_path,"conf/base"),
        os.path.join(project_path, "conf/local")
    ]
    conf_loader = ConfigLoader(conf_paths)
    credentials = conf_loader.get("credentials*",
                                  "credentials*/**")

    return credentials


credentials = get_credentials(project_path)
print(credentials['tpn'].keys())

dict_keys(['username', 'password'])


# 3. Retrieve Sample Data

In [26]:
# Importing class to retrievee data from TPN File Server.
from kedro_mlflow_tutorial.utils.tpn import TPNFileServer

tpn_client = TPNFileServer(
    credentials['tpn']['username'],
    credentials['tpn']['password'],
    server_ip,
    domain_name,
    shared_folder,
    sgs_base_path
)

session_id = 22 # SGS session ID
env_condition_id = 456 # SGS environmental condition ID

sgs_data = tpn_client.retrieveSGSData(
    session_id = session_id, 
    env_cond_id = env_condition_id
)

sgs_data.head()

2021-02-23 20:26:54,373 - SMB.SMBConnection - INFO - Authentication with remote machine "//10.1.1.52/" for user "rodrigo_cunha" will be using NTLM v2 authentication (with extended security)
2021-02-23 20:26:54,465 - SMB.SMBConnection - INFO - Now switching over to SMB2 protocol communication
2021-02-23 20:26:54,467 - SMB.SMBConnection - INFO - SMB2 dialect negotiation successful
2021-02-23 20:26:54,535 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'fcd198f09e53fe82'"
2021-02-23 20:26:54,536 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'fcd198f09e53fe82'"
2021-02-23 20:26:54,539 - SMB.SMBConnection - INFO - Server supports SMB signing
2021-02-23 20:26:54,541 - SMB.SMBConnection - INFO - SMB signing deactivated. SMB messages will NOT be signed.
2021-02-23 20:26:54,637 - SMB.SMBConnection - INFO - Authentication (on SMB2) successful!


/home/user01/.local/lib/python3.8/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


,x,y,z,xx,yy,zz,speed_x,speed_y,speed_z,speed_xx,...,rupture_time,session_id,swell_dir,swell_hs,swell_tp,wave_dir,wave_hs,wave_tp,win_dir,wind_speed
0,-0.300000,15.700000,-16.000000,0.000000,0.000000,244.500000,0.000000,0.000000,0.000000,0.000000,...,0.0,22,108.0,0.98,4.99,201.2,3.13,15.15,100.3,6.48
1,-0.312367,15.705600,-16.015366,-0.059015,0.018654,244.501190,0.010677,0.003775,-0.030110,-0.115982,...,0.0,22,108.0,0.98,4.99,201.2,3.13,15.15,100.3,6.48
2,-0.355629,15.723564,-16.058661,-0.226357,0.061959,244.509781,0.017714,-0.004206,-0.055346,-0.213925,...,0.0,22,108.0,0.98,4.99,201.2,3.13,15.15,100.3,6.48
3,-0.425700,15.748679,-16.123413,-0.473367,0.102663,244.521118,0.019346,-0.003715,-0.072912,-0.271486,...,0.0,22,108.0,0.98,4.99,201.2,3.13,15.15,100.3,6.48
4,-0.498472,15.766123,-16.198757,-0.747915,0.114695,244.519836,0.018630,0.011545,-0.075985,-0.267755,...,0.0,22,108.0,0.98,4.99,201.2,3.13,15.15,100.3,6.48


# 4. Create the download Node

Node specifications:

- Automate the data downloading from SGS
- Define SGS session and environmental condition IDs as parameters

In [5]:
import os
from typing import Dict, List, Iterable
from kedro_mlflow_tutorial.utils.tpn import TPNFileServer
from kedro.config import ConfigLoader


def download_sgs_data(credentials: Dict,
                     sgs_params: Dict,
                     tpn_params: Dict,
                     project_path: str,
                ) -> Dict:

    sgs_base_path = sgs_params['base_path']
    session_ids = sgs_params['download']['session_ids']
    env_condition_ids = sgs_params['download']['env_cond']['ids']
    mode = sgs_params['download']['mode']
    download_dir = sgs_params['download']['dir']
    file_only = sgs_params['download']['file_only']
    start_env_id = sgs_params['download']['env_cond']['start_id']
    end_env_id = sgs_params['download']['env_cond']['end_id']
    env_step = sgs_params['download']['env_cond']['step']

    server_ip = tpn_params['server_ip']
    domain_name = tpn_params['domain_name']
    shared_folder = tpn_params['shared_folder']


    iterable = define_download_data(
        session_ids = session_ids,
        env_ids = env_condition_ids,
        mode = mode,
        start_env_id = start_env_id,
        end_env_id = end_env_id,
        env_step = env_step,
    )

    data = {}
    for session_id, env_id in iterable:
        df, filename = download_sgs_file(
            session_id = session_id,
            env_condition_id = env_id,
            username = credentials['tpn']['username'],
            password = credentials['tpn']['password'],
            server_ip = server_ip,
            domain_name = domain_name,
            shared_folder = shared_folder,
            sgs_base_path = sgs_base_path,
            project_path = project_path,
            download_dir = download_dir
        )
        data[filename] = df

    return data


def define_download_data(
        session_ids: List[int],
        env_ids: List[int] = None,
        mode: str = 'std',
        start_env_id: int = None,
        end_env_id: int = None,
        env_step: int = None,
    ) -> Iterable:
    '''
    Returns an list of tuples (i,j) with i corresponding to the SGS session ID
    and j to the environment condition ID

    Parameters:

    Returns:

    '''

    iterable = []
    if mode == 'std' and env_ids:
        iterable = [(i,j) for i in session_ids
                            for j in env_id]

    elif (mode == 'sparse'
          and start_env_id is not None
          and end_env_id is not None
          and env_step is not None):
        iterable = [(i,j) for i in session_ids
                            for j in range(start_env_id,end_env_id,env_step)]

    return iterable


def download_sgs_file(session_id: int,
                    env_condition_id: int,
                    username: str,
                    password: str,
                    server_ip: str,
                    domain_name: str,
                    shared_folder: str,
                    sgs_base_path: str,
                    project_path: str,
                    download_dir: str = 'data/01_raw/sgs',
                    file_only: bool = True) -> str:

    tpn_client = TPNFileServer(
        username,
        password,
        server_ip,
        domain_name,
        shared_folder,
        sgs_base_path
    )

    if file_only:
        data_dir = download_dir
    else:
        data_dir = os.path.join(download_dir,'/{}/{}'.format(
                        session_id,
                        str(env_condition_id).zfill(4)
                    ))

    output_dir = os.path.join(
        project_path,
        data_dir
    )

    return tpn_client.downloadSGSDataAsDataFrame(
        session_id = session_id,
        env_cond_id = env_condition_id,
        output_dir = output_dir,
        file_only = file_only,
    )


In [6]:
sgs_data = download_sgs_data(
    credentials = credentials,
    sgs_params = catalog.load("params:sgs"),
    tpn_params = catalog.load("params:tpn"),
    project_path = catalog.load("params:project_path"),
)

2021-02-23 20:09:58,778 - kedro.io.data_catalog - INFO - Loading data from `params:sgs` (MemoryDataSet)...
2021-02-23 20:09:58,780 - kedro.io.data_catalog - INFO - Loading data from `params:tpn` (MemoryDataSet)...
2021-02-23 20:09:58,791 - kedro.io.data_catalog - INFO - Loading data from `params:project_path` (MemoryDataSet)...


  0%|          | 0/100 [00:00<?, ?it/s]

2021-02-23 20:09:58,827 - SMB.SMBConnection - INFO - Authentication with remote machine "//10.1.1.52/" for user "rodrigo_cunha" will be using NTLM v2 authentication (with extended security)
2021-02-23 20:09:58,923 - SMB.SMBConnection - INFO - Now switching over to SMB2 protocol communication
2021-02-23 20:09:58,924 - SMB.SMBConnection - INFO - SMB2 dialect negotiation successful
2021-02-23 20:09:58,986 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'c29e253d4352540b'"
2021-02-23 20:09:58,987 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'c29e253d4352540b'"
2021-02-23 20:09:58,990 - SMB.SMBConnection - INFO - Server supports SMB signing
2021-02-23 20:09:58,991 - SMB.SMBConnection - INFO - SMB signing deactivated. SMB messages will NOT be signed.
2021-02-23 20:09:59,068 - SMB.SMBConnection - INFO - Authentication (on SMB2) successful!


/home/user01/.local/lib/python3.8/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
  1%|          | 1/100 [00:01<01:52,  1.14s/it]

2021-02-23 20:09:59,963 - SMB.SMBConnection - INFO - Authentication with remote machine "//10.1.1.52/" for user "rodrigo_cunha" will be using NTLM v2 authentication (with extended security)
2021-02-23 20:10:00,065 - SMB.SMBConnection - INFO - Now switching over to SMB2 protocol communication
2021-02-23 20:10:00,067 - SMB.SMBConnection - INFO - SMB2 dialect negotiation successful
2021-02-23 20:10:00,153 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'6c8df0c69b97f4f1'"
2021-02-23 20:10:00,155 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'6c8df0c69b97f4f1'"
2021-02-23 20:10:00,158 - SMB.SMBConnection - INFO - Server supports SMB signing
2021-02-23 20:10:00,160 - SMB.SMBConnection - INFO - SMB signing deactivated. SMB messages will NOT be signed.
2021-02-23 20:10:00,217 - SMB.SMBConnection - INFO - Authentication (on SMB2) successful!


...

 98%|█████████▊| 98/100 [02:02<00:02,  1.20s/it]

2021-02-23 20:12:00,854 - SMB.SMBConnection - INFO - Authentication with remote machine "//10.1.1.52/" for user "rodrigo_cunha" will be using NTLM v2 authentication (with extended security)
2021-02-23 20:12:00,995 - SMB.SMBConnection - INFO - Now switching over to SMB2 protocol communication
2021-02-23 20:12:00,997 - SMB.SMBConnection - INFO - SMB2 dialect negotiation successful
2021-02-23 20:12:01,066 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'b385488fc7df9a73'"
2021-02-23 20:12:01,067 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'b385488fc7df9a73'"
2021-02-23 20:12:01,070 - SMB.SMBConnection - INFO - Server supports SMB signing
2021-02-23 20:12:01,071 - SMB.SMBConnection - INFO - SMB signing deactivated. SMB messages will NOT be signed.
2021-02-23 20:12:01,136 - SMB.SMBConnection - INFO - Authentication (on SMB2) successful!


 99%|█████████▉| 99/100 [02:03<00:01,  1.18s/it]

2021-02-23 20:12:01,982 - SMB.SMBConnection - INFO - Authentication with remote machine "//10.1.1.52/" for user "rodrigo_cunha" will be using NTLM v2 authentication (with extended security)
2021-02-23 20:12:02,076 - SMB.SMBConnection - INFO - Now switching over to SMB2 protocol communication
2021-02-23 20:12:02,077 - SMB.SMBConnection - INFO - SMB2 dialect negotiation successful
2021-02-23 20:12:02,142 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'3aa79d2ac7444430'"
2021-02-23 20:12:02,143 - SMB.SMBConnection - INFO - Performing NTLMv2 authentication (on SMB2) with server challenge "b'3aa79d2ac7444430'"
2021-02-23 20:12:02,146 - SMB.SMBConnection - INFO - Server supports SMB signing
2021-02-23 20:12:02,147 - SMB.SMBConnection - INFO - SMB signing deactivated. SMB messages will NOT be signed.
2021-02-23 20:12:02,227 - SMB.SMBConnection - INFO - Authentication (on SMB2) successful!


100%|██████████| 100/100 [02:04<00:00,  1.24s/it]


In [10]:
sgs_data.keys()

dict_keys(['22_0001_pos.csv', '22_0011_pos.csv', '22_0021_pos.csv', '22_0031_pos.csv', '22_0041_pos.csv', '22_0051_pos.csv', '22_0061_pos.csv', '22_0071_pos.csv', '22_0081_pos.csv', '22_0091_pos.csv', '22_0101_pos.csv', '22_0111_pos.csv', '22_0121_pos.csv', '22_0131_pos.csv', '22_0141_pos.csv', '22_0151_pos.csv', '22_0161_pos.csv', '22_0171_pos.csv', '22_0181_pos.csv', '22_0191_pos.csv', '22_0201_pos.csv', '22_0211_pos.csv', '22_0221_pos.csv', '22_0231_pos.csv', '22_0241_pos.csv', '22_0251_pos.csv', '22_0261_pos.csv', '22_0271_pos.csv', '22_0281_pos.csv', '22_0291_pos.csv', '22_0301_pos.csv', '22_0311_pos.csv', '22_0321_pos.csv', '22_0331_pos.csv', '22_0341_pos.csv', '22_0351_pos.csv', '22_0361_pos.csv', '22_0371_pos.csv', '22_0381_pos.csv', '22_0391_pos.csv', '22_0401_pos.csv', '22_0411_pos.csv', '22_0421_pos.csv', '22_0431_pos.csv', '22_0441_pos.csv', '22_0451_pos.csv', '22_0461_pos.csv', '22_0471_pos.csv', '22_0481_pos.csv', '22_0491_pos.csv', '22_0501_pos.csv', '22_0511_pos.csv', '

# 5. Next steps

- 1. Update **nodes.py** file for data integration pipeline
- 2. Update **pipeline.py** file for data integration pipeline
- 3. Update **hooks.py** file
- 4. Update **conf/base/catalog.yml** file
- 5. **Commit code to repo**